In [15]:
import pandas as pd
df=pd.read_csv('afterPP_dataset.csv')

In [17]:
df

,Reviews
0,كان لي تجربه اكثر من راءعه اليوم في هذا الفرع ...
1,اسوء خدمه شركه اتصالات علي المستوي العربي كله ...
2,اشكر الموظفه ضيء علي تعاملها ورقيها وخدمتها لي...
3,اسوا نت في العالم عانيت مليون مرررررا منهم رفع...
4,الفرع جدا ممتاز واشكر الاخوات الموظفات علي رحا...
...,...
382,التذكره ايش صار عليها محد فيكم تكرم تواصل معي ...
383,قويه
384,انترنت لا محدود سرعه ميقا زين السعوديه
385,سرعه عاليه ميجا ال زين يارب وماخاب من قال يارب


In [18]:
df.dropna(inplace=True)

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from nltk.corpus import stopwords
import nltk

# Assuming `df` is your DataFrame and it has a column named 'text' with Arabic documents
# df = pd.read_csv('your_dataset.csv') # Uncomment this line if you need to read the DataFrame from a CSV file

# Download the stopwords from NLTK
nltk.download('stopwords')

# Load Arabic stopwords
# Load Arabic stopwords and convert them to a list
arabic_stopwords = list(stopwords.words('arabic'))


# Create TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, max_features=1000, stop_words=arabic_stopwords)

# Fit and transform the documents
tfidf_matrix = tfidf_vectorizer.fit_transform(df['Reviews'])

# Use KMeans clustering
num_clusters = 3  # Change this number based on how many clusters you want to create
km = KMeans(n_clusters=num_clusters, random_state=42)
km.fit(tfidf_matrix)

# Assignments of documents to clusters
df['cluster'] = km.labels_

# Now your DataFrame has an additional column 'cluster' indicating the cluster assignment for each document

# You can inspect the first few rows to check the cluster assignments
df.head()

# Optionally, you can save this DataFrame to a new CSV file
# df.to_csv('clustered_dataset.csv', index=False)


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/shahadaleissa/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/Users/shahadaleissa/Downloads/Data Mining Project/.conda/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:408: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['آمين', 'أب', 'أخ', 'أفعل', 'أفعله', 'ؤلاء', 'إل', 'إم', 'ات', 'اتان', 'ارتد', 'ان', 'انفك', 'برح', 'تان', 'تبد', 'تحو', 'تعل', 'حد', 'حم', 'حي', 'خب', 'ذار', 'سيما', 'صه', 'ظل', 'ظن', 'عد', 'قط', 'مر', 'مكان', 'مكانكن', 'نب', 'هات', 'هب', 'واها', 'وراء'] not in stop_words.
  warnings.warn(


,Reviews,cluster
0,كان لي تجربه اكثر من راءعه اليوم في هذا الفرع ...,0
1,اسوء خدمه شركه اتصالات علي المستوي العربي كله ...,2
2,اشكر الموظفه ضيء علي تعاملها ورقيها وخدمتها لي...,0
3,اسوا نت في العالم عانيت مليون مرررررا منهم رفع...,0
4,الفرع جدا ممتاز واشكر الاخوات الموظفات علي رحا...,1


In [20]:
df[df['cluster'] == 1]

,Reviews,cluster
4,الفرع جدا ممتاز واشكر الاخوات الموظفات علي رحا...,1
8,ممتاز والموظفه اسلوبها جدا راقي وسلسه في التعامل,1
9,بطء جدا جدا جدا الخبره تكاد شبه معدومه,1
25,ممتاز جدا اشكر الموضفين علي حسن الاخلاق والتعاون,1
29,ممتاز,1
40,شركه ذات خدمه وتعامل رديءه جدا لا يتجاوبون معك...,1
41,فرع ممتاز استقبال الموظف كان ممتاز ماعليه زحمه,1
43,شركه اتصالات ممتازه و موقعها ممتاز يقدمون كل ا...,1
46,تعامل الموظف احمد السالم ممتاز جدا,1
50,التقييم علي خدمه الواتساب التعبانه,1
